In [ ]:
!pip install deeppavlov

In [ ]:
from deeppavlov import build_model, configs, train_model
from deeppavlov.core.commands.utils import parse_config
from deeppavlov.core.data.utils import get_all_elems_from_json
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
n = 5
data = pd.DataFrame(columns=['topic', 'text'])
for i in range(1, n+1):
  new_topic = pd.read_csv(f"./drive/MyDrive/requests_gpt_{i}.csv", header=None)
  new_topic.columns = ['topic', 'text']
  data = pd.concat([data, new_topic])
data.drop(data[data['topic'] == 0].index, inplace = True)
print(data['topic'].value_counts())
data

1    155
5    152
4    150
2    140
3    140
Name: topic, dtype: int64


,topic,text
0,1,Каким образом можно предъявить иск о задолженн...
1,1,Каковы основания для обращения с иском о задол...
2,1,Какие органы могут принимать и рассматривать и...
3,1,Какие последствия могут возникнуть в случае не...
4,1,Каково количество и размер просроченных платеж...
...,...,...
147,5,Какие последующие шаги следует предпринять пос...
148,5,Как учитывать прецеденты и практику судебных р...
149,5,Поставщик взял деньги и не поставил товар
150,5,Не поставил товар


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(data['text'], data['topic'], test_size = 0.3)
X_train, X_valid, y_train, y_valid = train_test_split(data['text'], data['topic'], test_size=0.15, random_state=54)
train_data = pd.concat([X_train, y_train], axis=1)
valid_data = pd.concat([X_valid, y_valid], axis=1)
# test_data = pd.concat([X_test, y_test], axis=1)
test_data = pd.concat([X_valid, y_valid], axis=1)
print(train_data['topic'].value_counts())

1    137
4    129
5    127
2    117
3    116
Name: topic, dtype: int64


In [ ]:
valid_data['topic'].value_counts()

1    12
5    12
3    11
4    11
2     7
Name: topic, dtype: int64

In [ ]:
test_data['topic'].value_counts()

2    42
5    33
1    27
3    25
4    25
Name: topic, dtype: int64

In [ ]:
valid_data.reset_index(drop = True, inplace = True)
train_data.to_csv("train.csv", index=False)
valid_data.to_csv("valid.csv", index=False)
test_data.to_csv("test.csv", index=False)

In [ ]:
model_config = parse_config('topics_distilbert_base_uncased')
#print(model_config)
# data_path = model_config['dataset_reader']['data_path']
# pd.read_csv(os.path.join(data_path, "train.csv"))
# model = build_model(model_config, download=True)
# print(model_config['chainer']['pipe'][3])
# requirements = set()
# for req in model_config.get('metadata', {}).get('requirements', []):
#     requirements.add(req)

# config_references = [expand_path(config_ref) for config_ref in get_all_elems_from_json(model_config, 'config_path')]
# requirements |= {req for config in config_references for req in get_config_requirements(config)}
# requirements

In [ ]:
model_config['dataset_reader']['data_path'] = "/content/"
# model_config['chainer']['pipe'][3]['load_path'] = "/content/"
# model_config['chainer']['pipe'][3]['save_path'] = "/content/"
# model_config['chainer']['pipe'][1]['load_path'] = "/content/classes.dict"
# model_config['chainer']['pipe'][1]['save_path'] = "/content/classes.dict"
model_config['chainer']['pipe'][3]['load_path'] = "/content/drive/MyDrive/model"
model_config['chainer']['pipe'][3]['save_path'] = "/content/drive/MyDrive/model"
model_config['chainer']['pipe'][1]['load_path'] = "/content/drive/MyDrive/classes.dict"
model_config['chainer']['pipe'][1]['save_path'] = "/content/drive/MyDrive/classes.dict"
model_config['chainer']['out'] = ['y_pred_labels', 'y_pred_probas']
model = train_model(model_config)
# model = build_model(model_config)

2024-04-13 18:13:04.893 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 104: [saving vocabulary to /content/drive/MyDrive/classes.dict]
INFO:deeppavlov.core.data.simple_vocab:[saving vocabulary to /content/drive/MyDrive/classes.dict]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2it [00:00,  4.26it/s]
2024-04-13 18:13:07.945 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best f1_macro of 0.2066
INFO:deeppavlov.core.trainers.nn_trainer:Initial best f1_macro of 0.2066


{"valid": {"eval_examples_count": 111, "metrics": {"f1_macro": 0.2066, "f1_weighted": 0.2308, "accuracy": 0.2252, "roc_auc": 0.492}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


INFO:train_report:{"valid": {"eval_examples_count": 111, "metrics": {"f1_macro": 0.2066, "f1_weighted": 0.2308, "accuracy": 0.2252, "roc_auc": 0.492}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
10it [00:06,  1.59it/s]
1it [00:00,  4.21it/s]

{"train": {"eval_examples_count": 64, "metrics": {"f1_macro": 0.258, "f1_weighted": 0.345, "accuracy": 0.2656, "roc_auc": 0.8166}, "time_spent": "0:00:08", "epochs_done": 1, "batches_seen": 10, "train_examples_seen": 626, "loss": 1.3147743344306946}}



INFO:train_report:{"train": {"eval_examples_count": 64, "metrics": {"f1_macro": 0.258, "f1_weighted": 0.345, "accuracy": 0.2656, "roc_auc": 0.8166}, "time_spent": "0:00:08", "epochs_done": 1, "batches_seen": 10, "train_examples_seen": 626, "loss": 1.3147743344306946}}
2it [00:00,  4.81it/s]
2024-04-13 18:13:14.965 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 206: Improved best f1_macro from 0.2066 to 0.2544
INFO:deeppavlov.core.trainers.nn_trainer:Improved best f1_macro from 0.2066 to 0.2544
2024-04-13 18:13:14.968 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 208: Saving model
INFO:deeppavlov.core.trainers.nn_trainer:Saving model
2024-04-13 18:13:14.971 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line 175: Saving model to /content/drive/MyDrive/model.pth.tar.
INFO:deeppavlov.core.models.torch_model:Saving model to /content/drive/MyDrive/model.pth.tar.


...


INFO:train_report:{"train": {"eval_examples_count": 626, "metrics": {"f1_macro": 1.0, "f1_weighted": 1.0, "accuracy": 1.0, "roc_auc": 1.0}, "time_spent": "0:00:03"}}
2it [00:00,  4.94it/s]

{"valid": {"eval_examples_count": 111, "metrics": {"f1_macro": 0.9807, "f1_weighted": 0.982, "accuracy": 0.973, "roc_auc": 0.9993}, "time_spent": "0:00:01"}}



INFO:train_report:{"valid": {"eval_examples_count": 111, "metrics": {"f1_macro": 0.9807, "f1_weighted": 0.982, "accuracy": 0.973, "roc_auc": 0.9993}, "time_spent": "0:00:01"}}
2it [00:00,  4.95it/s]

{"test": {"eval_examples_count": 111, "metrics": {"f1_macro": 0.9807, "f1_weighted": 0.982, "accuracy": 0.973, "roc_auc": 0.9993}, "time_spent": "0:00:01"}}



INFO:train_report:{"test": {"eval_examples_count": 111, "metrics": {"f1_macro": 0.9807, "f1_weighted": 0.982, "accuracy": 0.973, "roc_auc": 0.9993}, "time_spent": "0:00:01"}}
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model = build_model(model_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-04-10 12:25:33.670 WARNING in 'deeppavlov.core.models.torch_model'['torch_model'] at line 151: Init from scratch. Load path /content/drive/MyDrive/model.pth.tar does not exist.


In [ ]:
pred = model(["Как написать Заявление на открытие аккредитива в валюте РФ?"])
pred

[[['3']],
 array([[8.6193495e-06, 2.5066861e-06, 5.1775860e-06, 7.6302922e-06,
         9.9999154e-01]], dtype=float32)]

In [ ]:
pred = model(list(test_data['text']))[0]
pred = list([int(x[0]) if len(x) > 0 else 0 for x in pred])
pred.append(0)
y_test_old = y_test
y_test = pd.concat([y_test, pd.Series({99 : 0})])
pd.crosstab(y_test, pred)

col_0,0,1,2,3,4,5
row_0,,,,,,
0,1,0,0,0,0,0
1,2,24,0,0,0,0
2,0,0,33,0,0,0
3,0,0,0,35,0,0
4,1,1,0,0,27,1
5,0,0,0,0,0,28


In [ ]:
pred.pop()
y_test = y_test_old

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score
# print(type_of_target(list(y_test)))
# print(type_of_target(pred))
y_list = list(y_test)
print('accuracy',accuracy_score(y_list, pred))
print('precision',precision_score(y_list, pred, average='macro'))
print('recall',recall_score(y_list, pred, average='macro'))

accuracy 0.9671052631578947
precision 0.8209195402298851
recall 0.803846153846154


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_list,pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.96      0.92      0.94        26
           2       1.00      1.00      1.00        33
           3       1.00      1.00      1.00        35
           4       1.00      0.90      0.95        30
           5       0.97      1.00      0.98        28

    accuracy                           0.97       152
   macro avg       0.82      0.80      0.81       152
weighted avg       0.99      0.97      0.98       152



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_data